In [3]:
#!pip install torchvision

In [4]:
from aicspylibczi import CziFile
import czifile
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import cv2
import os
import imageio
import ffmpeg
import time
import pandas as pd
from cellpose import io, models
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import json
import glob
import itertools
from PIL import Image
import torch.nn.functional as F
from utils import *
cudnn.benchmark = True
plt.ion()   # interactive mode

In [5]:

class VideoDataMIP:
    def __init__(self, files):
        self.data = {
        }
        
        for category, num in files:
            print(f"Loading in MIP {num}")
            assert category == 'mip', "Can't load non Mip file"
            file = {}
            file['video'] = get_file(category, num)
            
            frames, shp = file['video'].read_image(C=0)
            frames = scale_img(frames.squeeze())
            file['frames'] = frames
            print(f"frames {num}: {frames.shape}")
            file['masks'] = binarize_video(frames)           
    
            self.data[num] = file    
    def extract_all_traces(self, file_num, sequence_length, hist_length=2):
        # hist length is how many frames of history
        frames, masks = self.data[file_num]['frames'], self.data[file_num]['masks']
        N = len(frames)
        s = 0
        all_traces = []
        all_videos = []
        for i in range(N // sequence_length):
            print(f"Extracting traces from {s}:{s+sequence_length}")
            data, videos = extract_traces(frames[s:s+sequence_length], masks[s:s+sequence_length], hist=hist_length)
            s += sequence_length
            all_traces = all_traces + data
            all_videos = all_videos + videos
        
        if(N % sequence_length > 0):
            data, videos = extract_traces(frames[-1*sequence_length:], masks[-1*sequence_length:], hist=hist_length)
            all_traces = all_traces + data
            all_videos = all_videos + videos        
        self.data[file_num]['traces'] = all_traces
        self.data[file_num]['trace_videos'] = all_videos
            

In [6]:
# fake_data = [
#     {
#         "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
#         "masks": [np.ones([100, 100]), np.ones([100, 100])],
#         "patch": [np.ones([100, 100]), np.ones([100, 100])]
#     },
#     {
#         "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
#         "masks": [np.ones([100, 100]), np.ones((100, 100))],
#         "patch": [np.ones([100, 100]), np.ones((100, 100))]
#     },
#     {
#         "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
#         "masks": [np.ones([100, 100]), np.ones((100, 100))],
#         "patch": [np.ones([100, 100]), np.ones((100, 100))]
#     },
#     {
#         "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
#         "masks": [np.ones([100, 100]), np.ones((100, 100))],
#         "patch": [np.ones([100, 100]), np.ones((100, 100))]
#     },
#     {
#         "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
#         "masks": [np.ones([100, 100]), np.ones((100, 100))],
#         "patch": [np.ones([100, 100]), np.ones((100, 100))]
#     },
#     {
#         "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
#         "masks": [np.ones([100, 100]), np.ones((100, 100))],
#         "patch": [np.ones([100, 100]), np.ones((100, 100))]
#     },
#     {
#         "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
#         "masks": [np.ones([100, 100]), np.ones((100, 100))],
#         "patch": [np.ones([100, 100]), np.ones((100, 100))]
#     },
#     {
#         "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
#         "masks": [np.ones([100, 100]), np.ones((100, 100))],
#         "patch": [np.ones([100, 100]), np.ones((100, 100))]
#     },
#     {
#         "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
#         "masks": [np.ones([100, 100]), np.ones((100, 100))],
#         "patch": [np.ones([100, 100]), np.ones((100, 100))]
#     },
#     {
#         "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
#         "masks": [np.ones([100, 100]), np.ones((100, 100))],
#         "patch": [np.ones([100, 100]), np.ones((100, 100))]
#     }
    
# ]

# fake_data[0]["masks"][1].flatten().shape

In [27]:
from skimage.measure import centroid
import skimage.measure as skm



box_shape = (180, 180) #TO DO: find the biggest box and set it to this
X = 10

class CellBoxMaskPatch(torch.utils.data.Dataset):
    #input will be a Directory name, function is TO DO
    def __init__(
        self,
        files, 
        X=X):
        # print("this is the new and imporcces")
        
        self.video_extractor = VideoDataMIP(files)

        # list of dictionaries, each dictionary is one cell tracked throughout a video
        #each cell has 3 data types in a dictionary
            #mask, box and patches

        for i in files:
            self.video_extractor.extract_all_traces(i[1], X)
        
        
        self.cell_dict = []

        for key in self.video_extractor.data:
            entry = self.video_extractor.data[key]["traces"]
            for cell in entry:
                patches = [np.array(p) for p in cell["patches"]]
                boxes = [np.array(b) for b in cell['boxes']]
                masks = [np.array(m) for m in cell['masks']]
                
                self.cell_dict.append((boxes, masks, patches)) #cell dict is a list of 3 types by sequence

        self.num_cells = len(self.cell_dict) #this is a list of how many sequences we have
              
    def __len__(self):
        return self.num_cells
        

    def get_centroids(self, boxes, masks):
        N = len(masks)
        res = []
        centroids = [skm.centroid(binary.astype(np.uint8)) for binary in masks]
        # print("centroids are", centroids)
        for i in range(N):
            c = centroids[i]
            ymin, xmin = boxes[i][:2]
            # print("didnt even get to res")
            res.append([xmin+c[0], ymin+c[1]])
            # print("results is ", type(res), "shape", np.array(res).shape)
        return(np.array(res) - res[0]) 
   
    def pad_arrays(self, array, pad_amt=200):
        # print("array shape 0 is", array.shape[0])
        # print("array shape 1", array.shape[1])


        # print("x would be", pad_amt-array.shape[0])
        # print("y would be", pad_amt-array.shape[1])
        pad_width = ((0, pad_amt - array.shape[0]), (0, pad_amt - array.shape[1]))
        # print("pad width is ",pad_width)

        padded_array = np.pad(array, pad_width, mode='constant')
        # print("pad array is is ",padded_array.shape)
        return padded_array

    def __getitem__(self, idx):
        cell_sequences = self.cell_dict[idx]  #this is the first sequence of 10 cells
        boxes = cell_sequences[0]
        masks = cell_sequences[1]
        patches = cell_sequences[2]


        for cell_mask_num in np.arange(len(masks)): #should be sequence length (10) masks
                
                #make masks 1 and 0s
                cell_time = np.array(masks[cell_mask_num], dtype=np.int32)
                cell_time = np.where(cell_time >= 0, cell_time, 1)

                #pad masks
                # print("cell_time shape, before padding", cell_time.shape)
                cell_time = self.pad_arrays(cell_time)
                # print("cell_time shape, after padding", cell_time.shape)
                masks[cell_mask_num] = cell_time

                #pad patches
                # print("cell_time PATCH shape, before padding", cell_time.shape)
                cell_time_patch = np.array(patches[cell_mask_num], dtype=np.int32)
                # print("cell_time PATCH shape, after padding", cell_time.shape)

                cell_time_patch = self.pad_arrays(cell_time_patch)

                patches[cell_mask_num] = cell_time_patch

        # print(boxes)

        centroids = self.get_centroids(boxes, masks)
        
        # print("shoudl be 10", len(patches))
        # print("after all, should be 250,250", patches[0].shape)

        # print("shoudl be 10 MASKSS", len(masks))
        # print("after all, should be 250,250 MASKS", masks[0].shape)

        return centroids, masks, patches

In [8]:
# from torch.utils.data import random_split

# root_dir = "/mnt/datadisk/"
# subdirs = ['FactinProcessed', 'FactinMIP']

# mip_video_files = [
#     ('mip', 3),
#     ('mip', 6),
#     ('mip', 9),
# ]

# mip_test_files = [('mip', 3)] #so that I dont keep crashing the kernel

# # file = root_dir + subdirs

# video_extractor = VideoDataMIP(mip_test_files)

#         # list of dictionaries, each dictionary is one cell tracked throughout a video
#         #each cell has 3 data types in a dictionary
#             #mask, box and patches


In [9]:
# files = [('mip', 3)]

# video_extractor = VideoDataMIP(files)

# # list of dictionaries, each dictionary is one cell tracked throughout a video
# #each cell has 3 data types in a dictionary
# #mask, box and patches
# X=10

# for i in files:
#     video_extractor.extract_all_traces(i[1], X)


In [10]:
# import skimage.measure as skm
        
        
# cell_dict = []

# for key in video_extractor.data:
#             entry = video_extractor.data[key]["traces"]
#             for cell in entry:
#                 # print("cell is of type", type(cell))
#                 sequence = []
#                 patches = [np.array(p) for p in cell["patches"]]
#                 boxes = [np.array(b) for b in cell['boxes']]
#                 masks = [np.array(m) for m in cell['masks']]
#                 # print("first patch", patches[0])
#                 # print("first patch as arraty", np.array(patches[0]))
#                 # print("first patch shape", np.array(patches[0]).shape)

#                 # print("first mask", masks[0])
#                 # print("first mask as arraty", np.array(masks[0]))
#                 # print("first mask SHAPPEPEEEE", np.array(masks[0]).shape)

#                 # for i in np.arange(len(patches)):
#                 # #     sequence.append({
#                 # #         "boxes": boxes[i],
#                 # #         "patches": patches[i],
#                 # #         "masks": masks[i]
#                 # #     })
#                 # #     print(sequence)
#                 #         sequence.append((boxes, patches, masks))
#                 #         # print("sequence should be ten lists of 3",len(sequence))
#                 #         # print("here is the first list of ten, should be 3 long", len(sequence[0]))
#                 cell_dict.append((boxes, masks, patches)) #cell dict is a list of 3 types by sequence

# num_cells = len(cell_dict) #this is a list of how many sequences we have


# cell_sequences = cell_dict[0]  #this is the first sequence of 10 cells
# boxes = cell_sequences[0]
# patches = cell_sequences[1]
# masks = cell_sequences[2]


# def get_centroids(self, cell_data):
#         boxes, masks = np.array(cell_data['boxes']), cell_data['masks']
#         N = len(masks)
#         res = []
#         centroids = [skm.centroid(binary.astype(np.uint8)) for binary in masks]
#         for i in range(N):
#             c = centroids[i]
#             ymin, xmin = boxes[i][:2]
#             res.append([xmin+c[0], ymin+c[1]])
#         return(np.array(res) - res[0,:]) 
   
# def pad_arrays(self, array, pad_amt=200):
#         print(pad_amt)
#         print("tyep", type(array))
#         print("array shape 0 is", array.shape[0])
#         print("array shape 1", array.shape[1])

#         print("x would be", pad_amt-array.shape[0])
#         print("y would be", pad_amt-array.shape[1])
#         pad_width = ((0, pad_amt - array.shape[0]), (0, pad_amt - array.shape[1]))
#         print(pad_width)

#         padded_array = np.pad(array, pad_width, mode='constant')
#         return padded_array


# #make the masks all 1's for all the cells
# for cell_mask_num in np.arange(len(masks)): #should be sequence length (10) masks
#         # print("cell_mask_num", cell_mask_num)
#         cell_time = np.array(masks[cell_mask_num], dtype=np.int32)
#         # print("this should be one mask", cell_time)
#         cell_time = np.where(cell_time >= 0, cell_time, 1)
#         # print("min and max", np.min(cell_time), np.max(cell_time))
#         print("cell time shape and cell time 1 ", cell_time.shape)
#         cell_time = pad_arrays(cell_time)
#         print("cell time shape and cell time 2", cell_time.shape)

#         masks[cell_mask_num] = cell_time


#         cell_time_patch = np.array(masks[cell_mask_num], dtype=np.int32)
#         cell_time_patch = pad_arrays(cell_time_patch)

#         masks[cell_mask_num] = cell_time_patch

In [11]:
# masks = [v['masks'] for v in video_extractor.data[3]['traces']]#this is a list of 10 masks


# import numpy as np

# def pad_arrays(array, padding_amt=200):
#     pad_width = ((0, 200 - array.shape[0]), (0, 200 - array.shape[1]))
#     padded_array = np.pad(array, pad_width, mode='constant')
#     return padded_array


# #get make all the values == 1

# #padding
# shapes = []
# for cell_mask in masks: #there are 67 cells, each cell is a list of 10 cells
#     for cell_time in cell_mask:
#         cell_time = np.array(cell_time, dtype=np.int32)
#         cell_time = np.where(cell_time < 0, cell_time, 1)
#         # print(cell_time.shape)
        
#         cell_time = pad_arrays(cell_time)
#         shapes.append(cell_time.shape)
#         #gonna pad to [180, 180], but can be changed after

    
# print(np.max(shapes))
# print(np.min(shapes))

In [12]:
# cell_time = np.array([[1, 2, 3], [4, 5, 6]])

# print(np.max(cell_time))

# cell_time = np.where(cell_time < 1, cell_time, 1)


# print(np.max(cell_time))



In [13]:
# from skimage.measure import centroid
# files = [('mip', 3)]
# video_extractor = VideoDataMIP(files)

# # test_patches = video_extractor.data[3]['traces'][0]['patches'] #this is a list of X paddings

# # #get centroids

# # centroids = ([centroid(t) for t in test_patches])
# # centroids

In [28]:
from torch.utils.data import random_split

mip_video_files = [
    ('mip', 3)
]

dataset = CellBoxMaskPatch(mip_video_files, 10) # file, S, T

train, eval, test = random_split(dataset, [0.4, 0.2, 0.4])

input_datasets = {}
input_datasets["train"] = train
input_datasets["eval"] = eval
input_datasets["test"] = test

Loading in MIP 3
Loading dicty_factin_pip3-03_MIP.czi with dims [{'X': (0, 474), 'Y': (0, 2048), 'C': (0, 2), 'T': (0, 90)}]
frames 3: (90, 2048, 474)
Extracting traces from 0:10
Extracting cell  0
Extracting cell  1
Extracting cell  2
Extracting cell  3
Extracting cell  4
Extracting cell  5
Extracting cell  6
Extracting traces from 10:20
Extracting cell  0
Extracting cell  1
Extracting cell  2
Extracting cell  3
Extracting cell  4
Extracting cell  5
Extracting cell  6
Extracting cell  7
Extracting traces from 20:30
Extracting cell  0
Extracting cell  1
Extracting cell  2
Extracting cell  3
Extracting cell  4
Extracting cell  5
Extracting cell  6
Extracting cell  7
Extracting cell  8
Extracting traces from 30:40
Extracting cell  0
Extracting cell  1
Extracting cell  2
Extracting cell  3
Extracting cell  4
Extracting cell  5
Extracting cell  6
Extracting cell  7
Extracting traces from 40:50
Extracting cell  0
Extracting cell  1
Extracting cell  2
Extracting cell  3
Extracting cell  4
Ex

In [15]:
# # timestep = [[0] for t in range(T)]
# # timestep

# # batch = ((np.array([1 , 2, 3]), 1, 2) , 3), ((np.array([1 , 2, 3]), 1, 2) , 3)
# # current_boxes = torch.stack([torch.tensor(b[0][0], dtype=torch.long) for b in batch], dim=0)
# # current_boxes
#         #current     #next   #boxes                     #masks
# batch  = [[1, 2, 3], [[[1,2, 3, 4], [1, 2, 3,4]], [[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1,2, 3]]]], [[1, 2, 3], [[[1,2, 3, 4], [1, 2, 3,4]], [[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1,2, 3]]]]



# next_boxes = torch.stack([torch.tensor(b[1][0], dtype=torch.long) for b in batch], dim=0)


In [59]:
def collate_fn(batch, mode_box, mode_mask, mode_patch):
    

    centroids = torch.stack([torch.tensor(b[0], dtype=torch.int) for b in batch], dim=0)
    print(centroids.shape)
    current_masks = torch.stack([torch.tensor(b[1], dtype=torch.long) for b in batch], dim=0)
    current_masks = current_masks.reshape([4, 10, 40000])
    print(current_masks.shape)
    current_patches = torch.stack([torch.tensor(b[2], dtype=torch.long) for b in batch], dim=0)
    current_patches = current_patches.reshape([4, 10, 40000])

    print(current_patches.shape)

    selected_tensors = []
    if mode_box:
        selected_tensors.append(centroids)
    if mode_mask:
        selected_tensors.append(current_masks)
    if mode_patch:
        selected_tensors.append(current_patches)

    # Concatenate selected tensors along the last dimension
    combined_tensor = torch.cat(selected_tensors, dim=-1)

    # Reshape to add the singleton dimension
    combined_tensor = combined_tensor #.unsqueeze(1)

    return combined_tensor, centroids
    
    

In [63]:
mode_box = True
mode_mask = True
mode_patch = False

dataloaders = {}
dataloaders['train'] = torch.utils.data.DataLoader(
    input_datasets['train'],
    batch_size=4,
    shuffle=True,
    num_workers=0,
    collate_fn=lambda batch: collate_fn(batch, mode_box, mode_mask, mode_patch)
)

dataloaders['test'] = torch.utils.data.DataLoader(
    input_datasets['test'],
    batch_size=4,
    shuffle=True,
    num_workers=0,
    collate_fn=lambda batch: collate_fn(batch, mode_box, mode_mask, mode_patch)
)

dataloaders['eval'] = torch.utils.data.DataLoader(
    input_datasets['eval'],
    batch_size=4,
    shuffle=True,
    num_workers=0,
    collate_fn=lambda batch: collate_fn(batch, mode_box, mode_mask, mode_patch)
)

In [64]:
for batch in dataloaders['train']:
    # input, output = batch
    # Process the batches as needed
    # print("Input Shape:", input.shape)
    # print("Output Shape:", output.shape)
    print("Input:", batch[0].shape, "Centroids", batch[1].shape)
    
    break  

torch.Size([4, 10, 2])
torch.Size([4, 10, 40000])
torch.Size([4, 10, 40000])
Input: torch.Size([4, 10, 40002]) Centroids torch.Size([4, 10, 2])


In [62]:
#create an LSTM RNN

#to create a pytorch LSTM 
#The first axis is the sequence itself
# the second indexes instances in the mini-batch
# third indexes elements of the input. 

# we would input T boxes
batch_size = 4

class LSTM(nn.Module):
    def __init__(self, T, hidden_dim, S, num_layers=2):
        #(input is )
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
                           #input_size, hidden_size, num_layers
        self.lstm = nn.LSTM(T, hidden_dim, num_layers=2, batch_first=True) #stacking 2 LSTMs

        # The linear layer that maps from hidden state space to output space
        self.fc = nn.Linear(hidden_dim, S)
    def forward(self, input):


        #h_0: tensor of shape (D∗num_layers,N,Hout​) containing the initial hidden
        # state for each element in the input sequence. Defaults to zeros if (h_0, c_0) is not provided.
        #D = 2 if bidirectional is True, 1 otherwise
        #num layers is the number of layers
        #N = size of input
        #Hout is projection size
        h0 = torch.zeros(self.num_layers, input.size(0), self.hidden_dim).to(input.device)
        c0 = torch.zeros(self.num_layers, input.size(0), self.hidden_dim).to(input.device)

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, _ = self.lstm(input, (h0.detach(), c0.detach()))

        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out
        lstm_out, h_n = self.lstm(input, batch_size, -1) #output and final hidden state
        output = self.hidden2out(lstm_out.view(len(input), -1))
        #output = F.log_softmax(output, dim=1)
        return output

In [20]:
#mip only had one zplane 
# Example input data
#parameters for mask and patch
data_dim =  1000 #TO DO: find the biggest box and set it to this
learning_rate = 0.0001
epochs = 25

T = 10 #len predictions
hidden_dims = 100
output_dim = input_dim

def sigmoid(x):
    return 1/ 1+ np.exp(-x)

batch_size = 3
seq_length = 4
input_dim = 4
hidden_dim = 5
num_layers = 1
#input is (batch, seq, feature)
input_data = torch.randn(batch_size, input_dim, input_dim)

# Create LSTM model
lstm_model = LSTM(input_dim, hidden_dim, output_dim, num_layers)

# Forward pass with desired sequence length T
T = seq_length
output = lstm_model(input_data)

print("Input shape:", input_data.shape)
print("Output shape:", output.shape)


NameError: name 'input_dim' is not defined

In [ ]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

input_dim = 0
if mode_box == True:
    input_dim +=  4 #(T, 4) for the bounding boxes
if mode_mask == True:
    input_dim += box_shape.flatten #flattened version of [H, W]
if mode_patch == True:
    input_dim += box_shape.flatten #flattened version of [H, W]

learning_rate = 0.0001
epochs = 25
T = 2
hidden_dims = 100
output_dim = input_dim
batch_size = 1
model = LSTM(input_dim, hidden_dims, output_dim)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# Do your training loop + eval, etc...
def train_model(model, criterion, optimizer, scheduler, num_epochs=epochs):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'dev']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:  #inputs shoudl be [B, S, D] #labes be a [B, T, D] batch, time, sequence shape
                inputs = inputs.to(device)

                 # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs, labels) #B, T, D

                    #LOSS AND PRED EDIT STUFF HERE AS WE TALKED ABOUT

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'dev' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())


    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model